In [1]:
import os
from importlib import reload

import h5py
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances

import utilities

reload(utilities)

<module 'utilities' from '/mnt/beegfs/projects/corrQuake/earthquakes/New_code/utilities.py'>

# Functions

In [2]:
def filter_spatio_temporally_close_mainshocks(mainshocks_days_dict, mainshocks_locations_dict, grid_size_km,
                                              min_days_between_mainshocks):
    """ Filter main shocks that are close in space and time, keeping only the one that happened first. """
    earth_radius_km = 6371  # km
    filtered_keys = set(mainshocks_days_dict.keys())  # Start with all keys
    to_remove = set()

    keys = sorted(mainshocks_days_dict.keys(), key=lambda x: mainshocks_days_dict[x])  # Sort keys by date

    # Compare each pair of main shocks
    n_keys = len(keys)
    print(f"Number of main shocks: {n_keys}")
    for i in range(n_keys - 1):
        for j in range(i + 1, n_keys):
            key_i, key_j = keys[i], keys[j]
            if key_i in to_remove or key_j in to_remove:
                continue  # Skip if already marked for removal

            day_i, day_j = mainshocks_days_dict[key_i], mainshocks_days_dict[key_j]
            loc_i, loc_j = mainshocks_locations_dict[key_i], mainshocks_locations_dict[key_j]

            # Calculate spatial distance
            distance = haversine_distances(np.radians(loc_i).reshape(-1, 2),
                                           np.radians(loc_j).reshape(-1, 2)) * earth_radius_km

            # Check if they are close in space and time
            if (distance <= np.sqrt(2) * grid_size_km / 2) and ((day_j - day_i) < np.timedelta64(
                    min_days_between_mainshocks, 'D')):
                to_remove.add(key_j)  # Remove the later one

    # Update filtered keys by removing those marked
    filtered_keys.difference_update(to_remove)
    print(f"Number of main shocks after filtering spatiotemporally close mainshocks: {len(filtered_keys)}")
    return filtered_keys


def filter_gps_stations(earthquakes, ngl_list, search_radius, min_mainshock_mag, min_stations_per_main_shock, region):
    """ 
    Filter GPS stations based on their distance to the main shocks 
    and the number of stations around each main shock.
    Morally, 'earthquakes' is the catalog, with labeled sequences (in 'seq_id')
    ngl_list: metadata of the GPS stations
    """
    stations_to_download = set()
    mainshocks_stations = {}
    mainshocks_days = {}
    main_shocks_locations = {}
    region_min_lat, region_max_lat, region_min_lon, region_max_lon = utilities.return_regions()[region]
    earth_radius_km = 6371  # km

    for id_seq, seq in earthquakes.groupby('seq_id'):
        if len(seq) <= 1:
            continue
        mainshock = seq[(seq['type'] == 1) & (seq['mag'] >= min_mainshock_mag) & (seq['lat'] >= region_min_lat) & (
                seq['lat'] <= region_max_lat) & (seq['lon'] >= region_min_lon) & (seq['lon'] <= region_max_lon)]
        if mainshock.empty:
            continue
        # max_radius = max(10 ** (mainshock.mag.values[0] / 2 - 0.79),
        #                 search_radius)  # We ensure a minimum search radius of search_radius km
        ## ngl_list[["lat", "lon"]] : position of the station, at no specific day, at ~1km precision (is enough for now)
        distances = haversine_distances(np.radians(ngl_list[["lat", "lon"]]),
                                        np.radians(mainshock[["lat", "lon"]].values))[:, 0]
        distances *= earth_radius_km
        ## distances : list of all distances station-mainshock
        mainshock_day = mainshock.day.values[0]
        mainshock_location = mainshock[['lat', 'lon']].values[0]
        main_shocks_locations[id_seq] = mainshock_location
        # We only consider stations that were active n_days_before_mainshock days before the main shock 
        # and n_days_after_mainshock days after and that are within the max_radius distance from the main shock
        valid_mask = (distances <= search_radius) & (
                ngl_list.begin.values <= (mainshock_day - np.timedelta64(n_days_before_mainshock, 'D'))) & (
                             ngl_list.end.values >= (mainshock_day + np.timedelta64(n_days_after_mainshock,
                                                                                    'D')))

        valid_stations = ngl_list.name[valid_mask]
        if len(valid_stations) >= min_stations_per_main_shock:
            stations_to_download.update(valid_stations)
            mainshocks_stations[id_seq] = list(valid_stations)
            mainshocks_days[id_seq] = mainshock_day

    ## filter mainshocks that are too close in space and time
    filtered_keys = filter_spatio_temporally_close_mainshocks(mainshocks_days, main_shocks_locations, grid_size_km,
                                                              min_days_between_mainshocks)
    mainshocks_stations = {k: mainshocks_stations[k] for k in filtered_keys}
    mainshocks_days = {k: mainshocks_days[k] for k in filtered_keys}
    return stations_to_download, mainshocks_stations, mainshocks_days


def download_and_filter_gps_data(stations_to_download, gps_data_output_path, mainshocks_stations, mainshocks_days,
                                 n_days_before_mainshock, n_days_after_mainshock, ngl_list,
                                 mainshocks_gps_stations_daily_positions_path=""):
    """
    Download and filter GPS data for the stations around the main shocks. 
    
    """
    if os.path.exists(mainshocks_gps_stations_daily_positions_path):
        mainshocks_gps_stations_daily_positions = joblib.load(mainshocks_gps_stations_daily_positions_path)
        return mainshocks_gps_stations_daily_positions
    mainshocks_gps_stations_daily_positions = {}  ## GPS data at all relevant days, for each given mainshock
    if not stations_to_download:
        raise ValueError("No stations to download.")

    # We calculate the median latitude and longitude of the stations to download to use as a reference point
    # of the region
    filtered_ngl_list = ngl_list[ngl_list.name.isin(stations_to_download)]
    center_lat = filtered_ngl_list['lat'].median()
    center_lon = filtered_ngl_list['lon'].median()
    #print(f"Center of the region: {center_lat}, {center_lon}")
    # sys.exit()
    ## median of all stations seen in all mainshocks, i.e. approximately center of Japan

    # Load or download data for each station once and store it in a dictionary
    ## TODO station_data: rename into station_daily_positions TODO!
    stations_daily_positions = {}
    for station in stations_to_download:
        file_path = os.path.join(gps_data_output_path, f"{station}.csv")
        if os.path.exists(file_path):
            ## TIODO: rename data to: one_station_daily_positions
            one_station_daily_positions = pd.read_csv(file_path, sep=" ", parse_dates=['date'])
            print(f"{station} data loaded.")
        else:
            try:
                one_station_daily_positions = utilities.get_ngl_gps_data(station, "tenv3")
                #                 haversine_distances(data_in_rads, (center_lat, center_lon)) ## TODO: consider computing 
                ## the distance with the exact (haversine) formula
                utilities.convert_lat_lon_to_km(one_station_daily_positions, center_lat, center_lon)
                ## data[km_lat] and data[km_lon] are now filled
                one_station_daily_positions.to_csv(file_path, sep=" ", index=False)
                print(f"{station} data downloaded and saved.")
            except Exception as e:
                raise Exception(f"Error downloading data for station {station}: {e}")

        stations_daily_positions[station] = one_station_daily_positions  ## temporary dict

    # Pre-calculate the date ranges for each main shock
    mainshock_date_ranges = {
        id_seq: (
            mainshocks_days[id_seq] - np.timedelta64(n_days_before_mainshock, 'D'),
            mainshocks_days[id_seq] + np.timedelta64(n_days_after_mainshock, 'D')
        )
        for id_seq in mainshocks_days.keys()
    }

    # Process each main shock
    for id_seq, stations in mainshocks_stations.items():
        start_date, end_date = mainshock_date_ranges[id_seq]
        directory_path = f"{gps_data_output_path}/Sequence_{id_seq}"
        os.makedirs(directory_path, exist_ok=True)

        for station in stations:
            if station not in stations_daily_positions:
                raise ValueError(f"Data for station {station} not found.")

            ## subdata: morally, the GPS data restricted to a window, for 1 mainshock
            ## TODO: reanme subdata into oneMS_restrictedWindow_dailyPosition
            one_station_daily_positions_restricted_window_path = f"{directory_path}/{station}_positions.csv"
            if os.path.exists(one_station_daily_positions_restricted_window_path):
                one_station_daily_positions_restricted_window = pd.read_csv(
                    one_station_daily_positions_restricted_window_path, sep=" ", parse_dates=['date'])
                ## TODO: Remove the code below, it's not necessary
                one_station_daily_positions_restricted_window = one_station_daily_positions_restricted_window[
                    (one_station_daily_positions_restricted_window['date'] >= start_date) & (
                            one_station_daily_positions_restricted_window['date'] <= end_date)]

                print(f"{station} subdata around main shock {id_seq} loaded.")
            else:
                one_station_daily_positions = stations_daily_positions[station]
                one_station_daily_positions_restricted_window = one_station_daily_positions[
                    (one_station_daily_positions['date'] >= start_date) & (
                            one_station_daily_positions['date'] <= end_date)]
                one_station_daily_positions_restricted_window = one_station_daily_positions_restricted_window.sort_values(
                    by='date')

                if (
                        len(one_station_daily_positions_restricted_window) == n_days_before_mainshock + 1 + n_days_after_mainshock) and np.isfinite(
                    one_station_daily_positions_restricted_window[
                        ['lat', 'lon', 'z', 'km_lat', 'km_lon']].values).all():
                    one_station_daily_positions_restricted_window.to_csv(
                        one_station_daily_positions_restricted_window_path, sep=" ", index=False)
                    print(f"{station} subdata around main shock {id_seq} saved.")
                else:
                    print(f"Data of station {station} around main shock {id_seq} is incomplete or contains NaNs.")
                    continue

            if id_seq not in mainshocks_gps_stations_daily_positions:
                mainshocks_gps_stations_daily_positions[id_seq] = []
            mainshocks_gps_stations_daily_positions[id_seq].append(one_station_daily_positions_restricted_window)

    joblib.dump(mainshocks_gps_stations_daily_positions, mainshocks_gps_stations_daily_positions_path)
    return mainshocks_gps_stations_daily_positions


#converted_first2 = []
#haversined2 = []
def process_and_save_main_shocks_data(mainshocks_gps_stations_daily_positions, mainshocks_days, earthquakes,
                                      min_after_shock_mag,
                                      after_shock_time_window,
                                      n_days_after_mainshock,
                                      hdf5_output_file_path,
                                      min_stations_per_main_shock):
    """ Process and save the main shocks data to the HDF5 file. """
    earth_radius_km = 6371  # km
    km_per_degree_lat = np.pi * earth_radius_km / 180.0
    for id_seq, ms_gps_stations_positions in mainshocks_gps_stations_daily_positions.items():
        #converted_first = []
        #haversined = []
        print(f"Processing main shock {id_seq}...")
        displacements = []
        stations_positions = []

        # Extract aftershocks information
        seq = earthquakes[earthquakes.seq_id == id_seq]  ## all EQs of that sequence
        mainshock = seq[seq['type'] == 1 & (seq['day'] == mainshocks_days[id_seq])]
        # Filter aftershocks
        aftershocks = seq[(seq['type'] == 2) & (
                seq['day'] > mainshocks_days[id_seq] + np.timedelta64(n_days_after_mainshock, 'D')) & (
                                  seq['day'] <= mainshocks_days[id_seq] + np.timedelta64(after_shock_time_window, 'D'))
                          & (seq['mag'] >= min_after_shock_mag)]
        ## TODO: make a plot of the time , with MS day as a marker, and AS that are kept as markers too, with ticks at each days
        if len(aftershocks) < 1: continue
        mainshock_location = mainshock[['lat', 'lon']].values[0]
        #         station_aftershocks_locations : TODO: rename into oneMS_aftershocksLocations
        one_ms_aftershocks_locations = aftershocks[['lat', 'lon']].values
        #         station_aftershocks_mags : TODO: rename into oneMS_aftershocksMags
        one_ms_aftershocks_mags = aftershocks['mag'].values

        # loop over all stations of a given main shock, and for each compute the displacements
        for gps_station_positions in ms_gps_stations_positions:
            station_position = \
                gps_station_positions[
                    gps_station_positions['date'] == mainshocks_days[id_seq] - np.timedelta64(1, 'D')][
                    ['lat', 'lon']].values[0]  ## position of the station at day -1 
            stations_positions.append(station_position)
            ## TODO: check if maybe, converting to km first induces more numerical errors
            ## OR, directly, replace this code with minus operators to compute distances,
            ## replace it with haversine distnance (between position at day 0 and at day 1)
            ## haversine (lat_station_day-1, lon_staiton_day-1, lat_station_day+1, lon_station_day+1)
            # dis_lat = (
            #         gps_station_positions['km_lat'].shift(-1) - gps_station_positions['km_lat'].shift(1)
            # ).dropna().values
            # 
            # dis_lon = (
            #         gps_station_positions['km_lon'].shift(-1) - gps_station_positions['km_lon'].shift(1)
            # ).dropna().values
            # 
            # dis_z = (
            #         gps_station_positions['z'].shift(-1) - gps_station_positions['z'].shift(1)
            # ).dropna().values

            dis_lat = (
                    gps_station_positions['lat'].shift(-1) - gps_station_positions['lat'].shift(1)
            ).dropna().values
            dis_lat *= km_per_degree_lat  ## convert to km
            origin_lat = gps_station_positions['lat'].median()
            km_per_degree_lon = km_per_degree_lat * np.cos(np.radians(origin_lat))
            dis_lon = (
                    gps_station_positions['lon'].shift(-1) - gps_station_positions['lon'].shift(1)
            ).dropna().values
            dis_lon *= km_per_degree_lon  ## convert to km
            dis_z = (
                    gps_station_positions['z'].shift(-1) - gps_station_positions['z'].shift(1)
            ).dropna().values

            # Stack the displacements into a single numpy array
            # Each column corresponds to a displacement dimension (lat, lon, z)
            station_displacements = np.stack((dis_lat, dis_lon, dis_z), axis=-1)
            #print(dis_lat)
            #print(dis_lon)
            #print(dis_z)
            #print(station_displacements)
            #sys.exit()
            if not np.isfinite(station_displacements).all(): continue
            displacements.append(station_displacements)

        #     tmp1 = np.radians(gps_station_positions[['lat', 'lon']].shift(-1).dropna().values)
        #     tmp2 = np.radians(gps_station_positions[['lat', 'lon']].shift(1).dropna().values)
        #     distances = 6371 * haversine_distances(tmp1[1].reshape(-1,2), tmp2[1].reshape(-1,2))
        #     converted_first.append(np.linalg.norm(displacements[0][:, 0:2], axis=1).flatten())
        #     haversined.append(distances.flatten())
        # converted_first2.append(converted_first)
        # haversined2.append(haversined)
        #sys.exit()
        if len(displacements) < min_stations_per_main_shock: continue  ##TODO: put 3 as a parameter
        # Save to HDF5
        with h5py.File(hdf5_output_file_path, 'a') as f:
            if str(id_seq) in f:
                del f[str(id_seq)]
            grp = f.create_group(str(id_seq))
            grp.attrs['main_shock_day'] = str(np.datetime_as_string(mainshocks_days[id_seq], unit='D'))
            grp.attrs['main_shock_magnitude'] = mainshock.mag.values[0]
            grp.attrs['main_shock_location'] = mainshock_location

            grp.create_dataset('gps_stations_displacements', data=np.array(displacements))
            grp.create_dataset('stations_positions', data=np.vstack(stations_positions))
            grp.create_dataset('aftershocks_magnitudes', data=one_ms_aftershocks_mags)
            grp.create_dataset('aftershocks_locations', data=one_ms_aftershocks_locations)

# Parameters

In [3]:
earthquakes_path = "custom_catalog.csv"  #Path to the earthquake catalog file
search_radius = 300  #Search radius of GPS stations in km
min_mainshock_mag = 6  #Minimum magnitude of mainshocks to consider
min_stations_per_main_shock = 3  #Minimum number of GPS stations around a mainshock to consider it
regression = True  #Whether to use regression or classification
if regression:
    min_after_shock_mag = 2.5  #Minimum magnitude of aftershocks to consider
else:
    min_after_shock_mag = 4
after_shock_time_window = 45  #Days after the mainshock to search for aftershocks
n_days_before_mainshock = 1  #Number of days before the mainshock to consider
n_days_after_mainshock = 1  #Number of days after the mainshock to consider
min_days_between_mainshocks = 30  #Minimum number of days between two mainshocks to consider them as separate events
grid_size_km = 250  # Size of the square grid in km 
# path to the hdf5 file to store the data for the main shocks
hdf5_output_file_path = f"Data/Displacements_min_mainshock_mag={min_mainshock_mag}_min_stations_per_main_shock={min_stations_per_main_shock}_regression={regression}_min_after_shock_mag={min_after_shock_mag}_after_shock_time_window={after_shock_time_window}_n_days_before_mainshock={n_days_before_mainshock}_n_days_after_mainshock={n_days_after_mainshock}_min_days_between_mainshocks={min_days_between_mainshocks}_grid_size_km={grid_size_km}.hdf5"
# path to the dictionary containing the GPS data for the stations around the main shocks
dict_mainshocks_gps_stations_daily_positions_path = "Data/dict_mainshocks_gps_stations_daily_positions.pkl"
region = 'Japan'  # Region of the main shocks

# Main

In [4]:
# utilities.initialize_hdf5(hdf5_output_file_path) # not necessary, the file is created when the first main shock is processed

## load catalog
earthquakes = pd.read_csv(earthquakes_path, sep=" ", parse_dates=['datetime'])
earthquakes.rename(columns={'datetime': 'day'}, inplace=True)
earthquakes['day'] = earthquakes.day.values.astype('datetime64[D]')
earthquakes.sort_values(by='day', inplace=True)
earthquakes.reset_index(drop=True, inplace=True)

## load list of stations (all available stations, many that are not used)
gps_data_output_path = "Data"  #Path to store the GPS data
ngl_list = utilities.get_ngl_stations(f'{gps_data_output_path}/ngl_list.csv')

ngl stations already downloaded


In [5]:
## detect which stations are relevant
stations_to_download, mainshocks_stations, mainshocks_days = filter_gps_stations(earthquakes, ngl_list,
                                                                                 search_radius, min_mainshock_mag,
                                                                                 min_stations_per_main_shock, region)

Number of main shocks: 319
Number of main shocks after filtering spatiotemporally close mainshocks: 179


In [6]:
## download stations' data (positions) and filter by day-windows
## if csv's are in the proper location, then no download is node and it's very fast
mainshocks_gps_stations_daily_positions = download_and_filter_gps_data(stations_to_download, gps_data_output_path,
                                                                       mainshocks_stations,
                                                                       mainshocks_days, n_days_before_mainshock,
                                                                       n_days_after_mainshock,
                                                                       ngl_list,
                                                                       dict_mainshocks_gps_stations_daily_positions_path)

G136 data downloaded and saved.
G115 data downloaded and saved.
G148 data downloaded and saved.
G174 data downloaded and saved.
J427 data downloaded and saved.
J192 data downloaded and saved.
J647 data downloaded and saved.
CHNT data downloaded and saved.
R008 data downloaded and saved.
J556 data downloaded and saved.
J301 data downloaded and saved.
J916 data downloaded and saved.
R011 data downloaded and saved.
Z202 data downloaded and saved.
J336 data downloaded and saved.
G001 data downloaded and saved.
TAIP data downloaded and saved.
J731 data downloaded and saved.
J010 data downloaded and saved.
J065 data downloaded and saved.
J165 data downloaded and saved.
G061 data downloaded and saved.
G209 data downloaded and saved.
J244 data downloaded and saved.
NAAO data downloaded and saved.
I071 data downloaded and saved.
J550 data downloaded and saved.
J421 data downloaded and saved.
STKW data downloaded and saved.
J068 data downloaded and saved.
I086 data downloaded and saved.
J555 dat

In [7]:
process_and_save_main_shocks_data(mainshocks_gps_stations_daily_positions, mainshocks_days, earthquakes,
                                  min_after_shock_mag,
                                  after_shock_time_window,
                                  n_days_after_mainshock,
                                  hdf5_output_file_path,
                                  min_stations_per_main_shock)

Processing main shock 57...
Processing main shock 69...
Processing main shock 89...
Processing main shock 90...
Processing main shock 103...
Processing main shock 104...
Processing main shock 108...
Processing main shock 109...
Processing main shock 114...
Processing main shock 118...
Processing main shock 151...
Processing main shock 153...
Processing main shock 155...
Processing main shock 156...
Processing main shock 158...
Processing main shock 161...
Processing main shock 163...
Processing main shock 172...
Processing main shock 174...
Processing main shock 176...
Processing main shock 177...
Processing main shock 179...
Processing main shock 181...
Processing main shock 182...
Processing main shock 187...
Processing main shock 194...
Processing main shock 198...
Processing main shock 199...
Processing main shock 200...
Processing main shock 203...
Processing main shock 204...
Processing main shock 206...
Processing main shock 207...
Processing main shock 213...
Processing main sh

In [8]:
#converted_first2[11]

In [9]:
#haversined2[11]